In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

In [14]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [10]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
train_generator = train_datagen.flow_from_directory(
                                                    TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                                                        VALIDATION_DATA_DIR,
                                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                        batch_size=BATCH_SIZE,
                                                        shuffle=False,
                                                        class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [16]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape =
    (IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [21]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

/tmp/ipykernel_35768/2507997108.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
8/8 [==============================] - 32s 4s/step - loss: 0.6595 - acc: 0.7160 - val_loss: 0.1628 - val_acc: 0.9520
Epoch 2/10
8/8 [==============================] - 28s 4s/step - loss: 0.2377 - acc: 0.8860 - val_loss: 0.0943 - val_acc: 0.9700
Epoch 3/10
8/8 [==============================] - 28s 4s/step - loss: 0.1453 - acc: 0.9460 - val_loss: 0.0807 - val_acc: 0.9720
Epoch 4/10
8/8 [==============================] - 28s 4s/step - loss: 0.1331 - acc: 0.9520 - val_loss: 0.0582 - val_acc: 0.9760
Epoch 5/10
8/8 [==============================] - 29s 4s/step - loss: 0.1138 - acc: 0.9440 - val_loss: 0.0568 - val_acc: 0.9740
Epoch 6/10
8/8 [==============================] - 28s 4s/step - loss: 0.0846 - acc: 0.9720 - val_loss: 0.0573 - val_acc: 0.9760
Epoch 7/10
8/8 [==============================] - 28s 4s/step - loss: 0.0939 - acc: 0.9560 - val_loss: 0.0605 - val_acc: 0.9780
Epoch 8/10
8/8 [==============================] - 28s 4s/step - loss: 0.0627 - acc: 0.9820 - val_loss: 0

In [22]:
model.save('model.h5')

In [23]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [25]:
img_path = './elisa.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

1/1 [==============================] - 0s 63ms/step
[[0.00260389 0.9973961 ]]
{'cat': 0, 'dog': 1}
